# Some definitions

## Native coroutine

A function defined with `async def`. You can delegate a native coroutine to another using the `await` keyword. Even if there's no `await` keyword in a function, it is still a native coroutine if it has a `async def`. The `await` keyword cannot be used outside a native coroutine.

## Classic coroutine

A generator function that consumes data through calls on the `.send()` function and read that data using the `yield` keyword. They delegate to another classic coroutine using the `yield from` expression. Cannot be controlled by `await` and are not supported by `asyncio` anymore

## Generator based coroutines

A generator function decorated by `@types.coroutine`. Introduced in Python 3.5, this decorator makes a generator function into a coroutine that can be used with the `await` keyword. It has been discontinued in Python 3.11

## Async generator

A generator function defined with `async def` that has a `yield` on its body. It returns a generator object that offers a `__anext__()`, which is coroutine method to get the next item.

In [8]:
# Example: domain search using asyncio

# Search for domains with a Python keyword + the `.dev` suffix

import asyncio
import socket
from keyword import kwlist, softkwlist

MAX_KEYWORD_LEN = 4
KEYWORDS = sorted(kwlist + softkwlist)

async def probe(domain: str) -> tuple[str, bool]:
    loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    names = (kw for kw in KEYWORDS if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    coros = [probe(domain) for domain in domains]
    for coro in asyncio.as_completed(coros):
        domain, found = await coro
        mark = '+' if found else ' '
        print(f'{mark} {domain}')

await main()

+ def.dev
+ and.dev
  is.dev
+ del.dev
+ not.dev
  elif.dev
+ try.dev
  with.dev
+ as.dev
  else.dev
  for.dev
  true.dev
+ in.dev
  _.dev
  if.dev
  pass.dev
  none.dev
  case.dev
+ from.dev
  or.dev


# Awaitable

The same way a `for` loop works with a iterable, the `await` call will work with a awaitable. Usually we will see the following awaitables:

- A native coroutine object: which is a function defined with `async def`
- A `asyncio.Task` , which we can get by passing a coroutine object to the `asyncio.create_task()` function.

However, not always we need to `await` for a `Task`. We can create it using the `asyncio.create_task()` and it will be scheduled to run.

Warning: always create a variable to hold the `Task` reference, because it might be deleted by the garbage collector otherwise, because the event loop use weak references to manage the tasks.

On the other, if we want to execute a function and wait until it's finished, we can use `await other_coro()`. (Question: where?)

Low level awaitables:
- A object that implements `__await__` and that returns a iterable. Example: `asyncio.Future`
- A object that was written in another language using the API Python/C
- A generator function that uses the `types.coroutine` (discontinued)



In [11]:
# Downloading flags with asyncio (same example of chapter 20)

# For now, without error handling (just the happy path)

from httpx import AsyncClient
POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    # save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1, follow_redirects=True)
    return resp.read()


async def download_many(cc_list: list[str]) -> int:
    # return asyncio.run(supervisor(cc_list))
    # The code above doesnt work inside the ipynb, so I've changed to
    # the code below and redefined the download many and main to
    # native coroutines as well
    return await supervisor(cc_list)

async def supervisor(cc_list: list[str]) -> int:
    async with AsyncClient() as client:
        to_do = [download_one(client, cc) for cc in sorted(cc_list)]
        res = await asyncio.gather(*to_do)

    return len(res)

from typing import Callable
async def main(downloader: Callable[[list[str]], int]) -> None:  # <13>
    await downloader(POP20_CC)

await main(download_many)

MX IN RU IR DE VN US ET JP TR NG FR BR EG ID PK PH CN BD CD 